In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bnlp.corpus.util import remove_stopwords


In [3]:
rstate=10

In [4]:
df = pd.read_csv("Bengali hate speech.csv")

In [5]:
import string 
def remove_punctuation_from_sentence(sentence):
    sentence_without_punctuation="".join([c for c in sentence if c not in string.punctuation])
    return sentence_without_punctuation

df['sentence without punctuation'] = df['sentence'].apply(lambda x:remove_punctuation_from_sentence(x))
df.head()

,sentence,hate,category,sentence without punctuation
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব


In [6]:
df['Removed Stopped word'] = df['sentence without punctuation'].apply(lambda x: remove_stopwords(x,stopwords))
df.head()

,sentence,hate,category,sentence without punctuation,Removed Stopped word
0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী,"[যত্তসব, পাপন, শালার, ফাজলামী]"
1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার,"[পাপন, শালা, রে, রিমান্ডে, দরকার]"
2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,"[জিল্লুর, রহমান, স্যারের, ছেলে, এতো, বড়, জারজ,..."
3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,"[শালা, লুচ্চা, পাঠার, মত, যায়]"
4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,"[তুই, শালা, গাজা, খাইছচ, ।, তুর, মার, হেডায়, খ..."


In [7]:
df['hate_speech']=df['Removed Stopped word'].apply(' '.join)

In [8]:
y=df['hate']

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['hate_speech'])
sequences = tokenizer.texts_to_sequences(df['hate_speech'])
x= pad_sequences(sequences, maxlen=100)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=rstate)

In [11]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=100))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy*100.0)

Epoch 1/40
750/750 [==============================] - 58s 75ms/step - loss: 0.3909 - accuracy: 0.8307 - val_loss: 0.3010 - val_accuracy: 0.8820
Epoch 2/40
750/750 [==============================] - 54s 72ms/step - loss: 0.1898 - accuracy: 0.9267 - val_loss: 0.3258 - val_accuracy: 0.8737
Epoch 3/40
750/750 [==============================] - 54s 72ms/step - loss: 0.1053 - accuracy: 0.9622 - val_loss: 0.3879 - val_accuracy: 0.8658
Epoch 4/40
750/750 [==============================] - 55s 74ms/step - loss: 0.0652 - accuracy: 0.9772 - val_loss: 0.4679 - val_accuracy: 0.8558
Epoch 5/40
750/750 [==============================] - 56s 75ms/step - loss: 0.0453 - accuracy: 0.9843 - val_loss: 0.5156 - val_accuracy: 0.8485
Epoch 6/40
750/750 [==============================] - 57s 75ms/step - loss: 0.0334 - accuracy: 0.9886 - val_loss: 0.5589 - val_accuracy: 0.8348
Epoch 7/40
750/750 [==============================] - 57s 76ms/step - loss: 0.0280 - accuracy: 0.9904 - val_loss: 0.7238 - val_accuracy:

In [14]:
model.fit(X_train, y_train, batch_size=32, epochs=40, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy*100.0)

Epoch 1/40
534/534 [==============================] - 51s 91ms/step - loss: 0.0024 - accuracy: 0.9986 - val_loss: 1.3452 - val_accuracy: 0.8352
Epoch 2/40
534/534 [==============================] - 52s 97ms/step - loss: 0.0023 - accuracy: 0.9988 - val_loss: 1.4408 - val_accuracy: 0.8278
Epoch 3/40
534/534 [==============================] - 53s 99ms/step - loss: 0.0025 - accuracy: 0.9987 - val_loss: 1.4220 - val_accuracy: 0.8352
Epoch 4/40
534/534 [==============================] - 52s 98ms/step - loss: 0.0028 - accuracy: 0.9985 - val_loss: 1.3565 - val_accuracy: 0.8307
Epoch 5/40
534/534 [==============================] - 53s 99ms/step - loss: 0.0023 - accuracy: 0.9987 - val_loss: 1.3984 - val_accuracy: 0.8392
Epoch 6/40
534/534 [==============================] - 52s 98ms/step - loss: 0.0028 - accuracy: 0.9987 - val_loss: 1.4053 - val_accuracy: 0.8317
Epoch 7/40
534/534 [==============================] - 51s 96ms/step - loss: 0.0021 - accuracy: 0.9989 - val_loss: 1.4038 - val_accuracy: